# Basics of TensorFlow

This tutorial starts out with very basic tensorflow examples and slowly works up.

In [3]:
import tensorflow as tf

## Starting Simple

Tensorflow constants and variables define a graph with symbolic expressions.

In [4]:
x = tf.constant(10)
y = tf.constant(5)

with tf.Session() as sess:
    result = sess.run(x + y)
    print(result)

15


If you look at results without being in the context of a session, you'll see
that expressions like `add`, `mul`, `sub`, `matmul`, etc. haven't been computed
yet.

In [5]:
x = tf.constant(10)
y = tf.constant(5)
z = tf.add(x, y)

print(z)

Tensor("Add:0", shape=(), dtype=int32)


But if you invoke an operation using run on a session object, it will execute
all steps necessary to compute that value.

In [6]:
with tf.Session() as sess:
    result = sess.run(z)
    print(result)

15


## Using Placeholders

Running constant operations isn't particularly exciting. But TensorFlow uses 'placeholder' variables
that we can use to feed in values as inputs to the computational graph.

In [7]:
x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)
z = tf.add(x, y)

with tf.Session() as sess:
    result = sess.run(z, feed_dict={x: 3, y: 8})
    print(result)

11


We can feed in different values each time we execute the graph.

In [8]:
with tf.Session() as sess:
    result = sess.run(z, feed_dict={x: 4, y: 12})
    print(result)

16


Moreover, we can use Python's control structures to execute a graph multiple times and feed
different values each time.

In [9]:
from itertools import product

with tf.Session() as sess:
    for a, b in product(range(4), range(5, 8)):
        result = sess.run(z, feed_dict={x: a, y: b})
        print(result)

5
6
7
6
7
8
7
8
9
8
9
10


## Matrix Multiplication

In [10]:
import numpy as np

In [11]:
x = tf.placeholder(tf.float32, shape=(2, 2))
y = tf.placeholder(tf.float32, shape=(2, 2))
z = tf.matmul(x, y)

_Note_: feed_dict values have to be variables because mutating them must be supported.

In [12]:
with tf.Session() as sess:
    a = np.array([[1, 2],
                  [3, 4]], dtype=np.float32)
    b = np.array([[1, 2],
                  [3, 4]], dtype=np.float32)  
    result = sess.run(z, feed_dict={y: a, x: b})
    print(result)

[[  7.  10.]
 [ 15.  22.]]


## A Simple Learning Example

We'll just train the network to learn how to scale and add!

In [54]:
f = lambda x: x * 0.5 + 0.5

In [55]:
def repeater(fn):
    while True:
        val = np.random.rand()
        yield fn(val), val 

In [56]:
scale_add = repeater(f)
next(scale_add)

(0.9348075086815784, 0.8696150173631567)

These are some global settings we can use to configure how we run and optimize our network.

In [57]:
training_epochs = 50000
learning_rate = 1e-1

This is a basic fitting function we'll use.

In [58]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

W = tf.Variable(np.random.randn(), name="weight")
b = tf.Variable(np.random.randn(), name="bias")

pred = tf.add(tf.mul(X, W), b)

We have to have a cost function and an optimizer that tries to change the weights and biases to
change the cost function in the way we define (e.g. reduce it).

In [59]:
cost = tf.reduce_sum(tf.pow(pred-y, 2))/(2*batch_size)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
init = tf.initialize_all_variables()

Then we run a loop that feeds in values to train the function.

In [60]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        train_X, train_y = next(scale_add)
      
        sess.run(optimizer, feed_dict={X: train_X, y: train_y})
        current_cost = sess.run(cost, feed_dict={X: train_X, y: train_y})

        if not (epoch % 1000):
            print("Epoch:",  '%04d' % (epoch+1),
                  "Cost:",   "{:.9f}".format(current_cost),
                  "Weight:", sess.run(W),
                  "Bias:",   sess.run(b))
        
    test_val = 0.5
    print("Predicted: ", sess.run(pred, feed_dict={X: test_val}))
    print("Truth: ", f(test_val))

Epoch: 0001 Cost: 0.068217173 Weight: -1.19101 Bias: -2.25781
Epoch: 1001 Cost: 0.003659250 Weight: 0.213687 Bias: -0.42398
Epoch: 2001 Cost: 0.000649208 Weight: 0.530257 Bias: -0.0469852
Epoch: 3001 Cost: 0.000006252 Weight: 0.614598 Bias: 0.0256352
Epoch: 4001 Cost: 0.000034951 Weight: 0.645897 Bias: 0.0281774
Epoch: 5001 Cost: 0.000115528 Weight: 0.662968 Bias: 0.0133501
Epoch: 6001 Cost: 0.000238740 Weight: 0.682068 Bias: 0.00156094
Epoch: 7001 Cost: 0.000061830 Weight: 0.695405 Bias: -0.0176114
Epoch: 8001 Cost: 0.000014211 Weight: 0.718151 Bias: -0.0231903
Epoch: 9001 Cost: 0.000071009 Weight: 0.73523 Bias: -0.0359473
Epoch: 10001 Cost: 0.000479104 Weight: 0.747758 Bias: -0.055228
Epoch: 11001 Cost: 0.000058505 Weight: 0.770236 Bias: -0.0585728
Epoch: 12001 Cost: 0.000153796 Weight: 0.779543 Bias: -0.0797376
Epoch: 13001 Cost: 0.000058967 Weight: 0.795258 Bias: -0.093378
Epoch: 14001 Cost: 0.000106902 Weight: 0.818303 Bias: -0.0962058
Epoch: 15001 Cost: 0.000325009 Weight: 0.8314